In [3]:
from tensorflow.python.keras.utils import to_categorical
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras.layers import Input, RepeatVector,Dense,Bidirectional,Embedding,GRU,TimeDistributed,LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords 
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

C:\Users\dnishimoto.BOISE\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dnishimoto.BOISE\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dnishimoto.BOISE\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'

In [4]:
df=pd.read_csv("Reviews.csv",nrows=20)
print(df.columns)
df.drop_duplicates(subset=['Text'],inplace=True)  #dropping duplicates\n",
df.dropna(axis=0,inplace=True)

detail_sentences=[]
summary_sentences=[]
for detail,summary in zip(df['Text'],df['Summary']):
    print("Review:",detail,"\n\tSummary:",summary,"\n")
    detail_sentences.append(detail)
    summary_sentences.append(summary)
    
data=[]

for sent in df['Text']:
    # Add sos and eos tokens using string.join
    sent_new = " ".join(['sos', sent, 'eos'])
    data.append(sent_new)

summary=[]
for sent in df['Summary']:
    # Add sos and eos tokens using string.join
    sent_new = " ".join(['sos', sent, 'eos'])
    print(sent_new)
    summary.append(sent_new)


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
Review: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most. 
	Summary: Good Quality Dog Food 

Review: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo". 
	Summary: Not as Advertised 

Review: This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I

In [5]:
en_tokenizer = Tokenizer()
en_tokenizer.fit_on_texts(list(data))

x_data    =   en_tokenizer.texts_to_sequences(data) 
x_data    =   pad_sequences(x_data,  maxlen=None, padding='post')
#reverse the sequence
x_data=x_data[:,::-1]
en_vocab=len(en_tokenizer.word_index)+1

#print(en_tokenizer.index_word)

fr_tokenizer = Tokenizer()
fr_tokenizer.fit_on_texts(list(summary))

y_summary    =   fr_tokenizer.texts_to_sequences(summary) 
y_summary    =   pad_sequences(y_summary, maxlen=None, padding='post')

fr_vocab=len(fr_tokenizer.word_index)+1

max_len_text=x_data.shape[1]
max_len_summary=y_summary.shape[1]

#[[print(fr_tokenizer.index_word[i]) for i in sentence if i in fr_tokenizer.index_word] for sentence in y_summary]
for sentence_index in y_summary:
    sentence=""
    for i in sentence_index:
        if i in fr_tokenizer.index_word:
            sentence+=" "+fr_tokenizer.index_word[i]
    print(sentence)
print(y_summary)

X_train,X_test,y_train,y_test=train_test_split(x_data,y_summary,test_size=0.1,random_state=0,shuffle=True)

 sos good quality dog food eos
 sos not as advertised eos
 sos delight says it all eos
 sos cough medicine eos
 sos great taffy eos
 sos nice taffy eos
 sos great just as good as the expensive brands eos
 sos wonderful tasty taffy eos
 sos yay barley eos
 sos healthy dog food eos
 sos the best hot sauce in the world eos
 sos my cats love this diet food better than their regular food eos
 sos my cats are not fans of the new food eos
 sos fresh and greasy eos
 sos strawberry twizzlers yummy eos
 sos lots of twizzlers just what you expect eos
 sos poor taste eos
 sos love it eos
 sos great sweet candy eos
 sos home delivered twizlers eos
[[ 1  8 18  9  3  2  0  0  0  0  0  0  0]
 [ 1 10  5 19  2  0  0  0  0  0  0  0  0]
 [ 1 20 21 11 22  2  0  0  0  0  0  0  0]
 [ 1 23 24  2  0  0  0  0  0  0  0  0  0]
 [ 1  6  7  2  0  0  0  0  0  0  0  0  0]
 [ 1 25  7  2  0  0  0  0  0  0  0  0  0]
 [ 1  6 12  5  8  5  4 26 27  2  0  0  0]
 [ 1 28 29  7  2  0  0  0  0  0  0  0  0]
 [ 1 30 31  2  0  0  

In [6]:
hsize = 256
# Define an input layer
en_inputs = keras.layers.Input(shape=(max_len_text,en_vocab),name='encoder_inputs')
#en_inputs = keras.layers.Input(shape=(en_len,en_vocab),name='encoder_inputs')
# Define a GRU layer which returns the state
en_gru = keras.layers.GRU(hsize, return_state=True)
# Get the output and state from the GRU
en_out,en_state = en_gru(en_inputs)
# Define and print the model summary
encoder = keras.models.Model(inputs=en_inputs, outputs=en_state)

# Define a RepeatVector layer
de_inputs = RepeatVector(max_len_summary)(en_state)
# Define a GRU model that returns all outputs
decoder_gru = keras.layers.GRU(hsize, return_sequences=True)
# Get the outputs of the decoder
gru_outputs = decoder_gru(de_inputs, initial_state=en_state)
# Define a model with the correct inputs and outputs


de_dense = keras.layers.Dense(fr_vocab, activation='softmax')
de_dense_time = keras.layers.TimeDistributed(de_dense)
de_pred=de_dense_time(gru_outputs)

#enc_dec = Model(inputs=en_inputs, outputs=gru_outputs)
enc_dec = Model(inputs=en_inputs, outputs=de_pred)
enc_dec.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])


print(encoder.summary())
print(enc_dec.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_inputs (InputLayer)  (None, 152, 458)          0         
_________________________________________________________________
gru (GRU)                    [(None, 256), (None, 256) 549120    
Total params: 549,120
Trainable params: 549,120
Non-trainable params: 0
_________________________________________________________________
None
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, 152, 458)     0                                            
__________________________________________________________________________________________________
gru (GRU)                       [(None, 256), (None, 549120

In [8]:
def sents2seqs(input_type, sentences, tokenizer,sentence_len,vocab_size, onehot=False, pad_type='post', reverse=False):     
    encoded_text = tokenizer.texts_to_sequences(sentences)
    preproc_text = pad_sequences(encoded_text, padding=pad_type, truncating='post', maxlen=sentence_len)
    if reverse:
      # Reverse the text using numpy axis reversing
      preproc_text = preproc_text[:,::-1]
    if onehot:
        preproc_text = to_categorical(preproc_text, num_classes=vocab_size)
    return preproc_text

#n_epochs=10
#data_size=len(x_data)
#batch_size=1024

#for ei in range(n_epochs):
#    print("epoch",ei)
#    for i in range(0,data_size,batch_size):
#        en_x=to_categorical(X_train[i:i+batch_size],en_vocab)
#        fr_y=to_categorical(y_train[i:i+batch_size],fr_vocab)
#        enc_dec.train_on_batch(en_x,fr_y)


In [9]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['acc'])
    
    print(model.summary())
    return model


In [10]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

def preprocess_embedding(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk


preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess_embedding(detail_sentences, summary_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)
print("Record Count",len(detail_sentences))

Data Preprocessed
Max English sentence length: 150
Max French sentence length: 11
English vocabulary size: 455
French vocabulary size: 60
Record Count 20


In [12]:
tmp_x = pad(preproc_english_sentences)
model = model_final(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

#model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 128)          58368     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               591360    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 11, 512)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 11, 512)           1181184   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 11, 61)            31293     
Total params: 1,862,205
Trainable params: 1,862,205
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
20/20 [==============================] - 6s 294ms/sample - loss: 4.1130 - acc: 0.0045
Epoch 2/5
20/20 [==============

In [39]:


en_st="sos I have bought several of the Vitality canned dog food products and have found them all to be of good quality.The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most. eos"
#en_seq=sents2seqs('source',en_st,english_tokenizer,max_english_sequence_length,english_vocab_size,onehot=True,reverse=True)
sentence=[english_tokenizer.word_index[word] for word in en_st.split(" ") if word in english_tokenizer.word_index]
print(sentence)
sentence = pad_sequences([sentence], maxlen=max_english_sequence_length, padding='post')
#print(max_english_sequence_length)
sentences = np.array([sentence[0], preproc_english_sentences[0]])
#print(sentences)
fr_predictions=model.predict(sentences,len(sentences))
#print(french_tokenizer.index_word)
#print(fr_predictions)

for x in fr_predictions[0]:
        index=np.argmax(x)
        if index !=0:
            print(french_tokenizer.index_word[index])
#print(' '.join([french_tokenizer.index_word[np.argmax(x)] for x in fr_predictions[0]]))

[21, 61, 62, 11, 1, 64, 22, 33, 65, 2, 21, 34, 66, 35, 16, 67, 11, 13, 17, 69, 70, 71, 4, 72, 36, 4, 73, 74, 2, 5, 75, 6, 77, 2, 78, 79, 3, 17, 37, 36]
good
quality
dog
food
